In [1]:
# Importing the libraries
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [2]:
# Define paths to style, content, and output folders
style_image_path = 'Style/3.jpg'
content_image_path = 'Content/6.jpg'
output_folder = 'Output/'

In [3]:
# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [4]:
# Define a function to preprocess the input images
def preprocess_image(image_path):
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return img

In [5]:
# Define a function to deprocess the generated image
def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [6]:
# Load and preprocess the style and content images
style_image = preprocess_image(style_image_path)
content_image = preprocess_image(content_image_path)

In [7]:
# Create a VGG19 model with the desired layers for feature extraction
base_model = VGG19(weights='imagenet', include_top=False)
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
content_layer = 'block4_conv2'

In [8]:
# Create models for style and content feature extraction
style_extractor = Model(inputs=base_model.input, outputs=[base_model.get_layer(layer).output for layer in style_layers])
content_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer(content_layer).output)

In [9]:
# Define style and content target values
style_targets = style_extractor(style_image)
content_target = content_extractor(content_image)

In [10]:
# Define variables for the generated image and set them as trainable
generated_image = tf.Variable(content_image, dtype=tf.float32)
optimizer = Adam(learning_rate=2.0)

In [11]:
# Define a function to calculate the style loss
def style_loss(style_targets, style_outputs):
    style_loss = 0
    for target, output in zip(style_targets, style_outputs):
        target_gram = tf.linalg.einsum('bijc,bijd->bcd', target, target)
        output_gram = tf.linalg.einsum('bijc,bijd->bcd', output, output)
        style_loss += tf.reduce_mean(tf.square(target_gram - output_gram))
    return style_loss

In [12]:
# Define a function to calculate the content loss
def content_loss(content_target, content_output):
    return tf.reduce_mean(tf.square(content_target - content_output))

In [13]:
# Define a function to calculate the total variation loss
def total_variation_loss(x):
    a = tf.square(x[:, :-1, :-1, :] - x[:, 1:, :-1, :])
    b = tf.square(x[:, :-1, :-1, :] - x[:, :-1, 1:, :])
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [14]:
# Define the total loss as a combination of style, content, and total variation loss
total_variation_weight = 1e-6
style_weight = 1e-2
content_weight = 1e4

In [15]:
# Define a function to compute the total loss for the style transfer model
def total_loss(generated_image, style_targets, content_target):
    # Extract style features from the generated image using the style extractor model
    style_outputs = style_extractor(generated_image)
    
    # Extract content features from the generated image using the content extractor model
    content_output = content_extractor(generated_image)
    
    # Compute the total loss as a combination of style loss, content loss, and total variation loss
    # The weights (style_weight, content_weight, total_variation_weight) control the contribution of each loss component
    
    # Style loss measures the difference between the style of the generated image and the target style
    style_loss_value = style_loss(style_targets, style_outputs)
    
    # Content loss measures the difference between the content of the generated image and the target content
    content_loss_value = content_loss(content_target, content_output)
    
    # Total variation loss encourages spatial smoothness in the generated image
    tv_loss_value = total_variation_loss(generated_image)
    
    # Combine the individual losses with their respective weights to get the total loss
    total_loss_value = (
        style_weight * style_loss_value +
        content_weight * content_loss_value +
        total_variation_weight * tv_loss_value
    )
    
    return total_loss_value


In [16]:
# Create a function to train the style transfer model and save images after specified epochs
def train(style_targets, content_target, generated_image, epochs, save_interval):
    # Initialize a step counter
    step = 0
    
    # Iterate through the specified number of epochs
    for epoch in range(epochs):
        # Use a gradient tape to record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # Calculate the total loss for the current generated image
            loss = total_loss(generated_image, style_targets, content_target)
        
        # Compute the gradients of the loss with respect to the generated image
        grads = tape.gradient(loss, generated_image)
        
        # Update the generated image using the optimizer and computed gradients
        optimizer.apply_gradients([(grads, generated_image)])
        
        # Check if the current epoch is a multiple of the specified save interval
        if (epoch + 1) % save_interval == 0:
            # Save the generated image
            generated_img_np = deprocess_image(generated_image.numpy()[0])
            save_path = os.path.join(output_folder, f'{os.path.splitext(os.path.basename(content_image_path))[0]}_{epoch + 1}.jpg')
            tf.keras.preprocessing.image.save_img(save_path, generated_img_np)

        # Print the training progress (epoch, total loss)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")
        
        # Increment the step counter
        step += 1


In [17]:
# Set the number of training epochs
epochs = 300

# Set the interval at which to save the generated image during training
save_interval = 10


In [18]:
# Train the model
train(style_targets, content_target, generated_image, epochs, save_interval)

Epoch 1/1000, Loss: 1.6628513580693586e+18
Epoch 2/1000, Loss: 1.6156413524296335e+18
Epoch 3/1000, Loss: 1.563843634523013e+18
Epoch 4/1000, Loss: 1.5097512358492242e+18
Epoch 5/1000, Loss: 1.454097118347133e+18
Epoch 6/1000, Loss: 1.396679659071996e+18
Epoch 7/1000, Loss: 1.337596989436592e+18
Epoch 8/1000, Loss: 1.277154361356583e+18
Epoch 9/1000, Loss: 1.2157169501313434e+18
Epoch 10/1000, Loss: 1.1537255224846582e+18
Epoch 11/1000, Loss: 1.0917066757667553e+18
Epoch 12/1000, Loss: 1.0301960095793152e+18
Epoch 13/1000, Loss: 9.697096759121019e+17
Epoch 14/1000, Loss: 9.106813633827963e+17
Epoch 15/1000, Loss: 8.534830505189704e+17
Epoch 16/1000, Loss: 7.9840136625809e+17
Epoch 17/1000, Loss: 7.456549759751291e+17
Epoch 18/1000, Loss: 6.95381249904083e+17
Epoch 19/1000, Loss: 6.476639570870927e+17
Epoch 20/1000, Loss: 6.02544604088107e+17
Epoch 21/1000, Loss: 5.6004930430828544e+17
Epoch 22/1000, Loss: 5.201973335608525e+17
Epoch 23/1000, Loss: 4.83018000702636e+17
Epoch 24/1000, Lo

Epoch 190/1000, Loss: 3.592096757003059e+16
Epoch 191/1000, Loss: 3.5781153499643904e+16
Epoch 192/1000, Loss: 3.5642673016602624e+16
Epoch 193/1000, Loss: 3.550548746620109e+16
Epoch 194/1000, Loss: 3.5369618323275776e+16
Epoch 195/1000, Loss: 3.5235046260473856e+16
Epoch 196/1000, Loss: 3.5101711148253184e+16
Epoch 197/1000, Loss: 3.4969585069326336e+16
Epoch 198/1000, Loss: 3.4838676613627904e+16
Epoch 199/1000, Loss: 3.4708914914197504e+16
Epoch 200/1000, Loss: 3.458038157541376e+16
Epoch 201/1000, Loss: 3.4453042237538304e+16
Epoch 202/1000, Loss: 3.4326864688316416e+16
Epoch 203/1000, Loss: 3.420179953562419e+16
Epoch 204/1000, Loss: 3.4077801682305024e+16
Epoch 205/1000, Loss: 3.395493984783565e+16
Epoch 206/1000, Loss: 3.383319470486323e+16
Epoch 207/1000, Loss: 3.371250827132928e+16
Epoch 208/1000, Loss: 3.359286336736461e+16
Epoch 209/1000, Loss: 3.3474229928198144e+16
Epoch 210/1000, Loss: 3.335656070918963e+16
Epoch 211/1000, Loss: 3.3239851415371776e+16
Epoch 212/1000, Los

Epoch 375/1000, Loss: 2.1447846905511936e+16
Epoch 376/1000, Loss: 2.1399992379899904e+16
Epoch 377/1000, Loss: 2.135227958820864e+16
Epoch 378/1000, Loss: 2.1304706382954496e+16
Epoch 379/1000, Loss: 2.125725128930099e+16
Epoch 380/1000, Loss: 2.1209929339633664e+16
Epoch 381/1000, Loss: 2.1162757713821696e+16
Epoch 382/1000, Loss: 2.111573426438144e+16
Epoch 383/1000, Loss: 2.10688546963456e+16
Epoch 384/1000, Loss: 2.1022108272295936e+16
Epoch 385/1000, Loss: 2.097549069726515e+16
Epoch 386/1000, Loss: 2.092899767628595e+16
Epoch 387/1000, Loss: 2.088263565180928e+16
Epoch 388/1000, Loss: 2.083639818138419e+16
Epoch 389/1000, Loss: 2.079029600242893e+16
Epoch 390/1000, Loss: 2.074431837752525e+16
Epoch 391/1000, Loss: 2.06984674541568e+16
Epoch 392/1000, Loss: 2.065274752729088e+16
Epoch 393/1000, Loss: 2.0607128532156416e+16
Epoch 394/1000, Loss: 2.0561614763720704e+16
Epoch 395/1000, Loss: 2.051620836946739e+16


KeyboardInterrupt: 